In [3]:
import os
import pandas as pd

# Define the paths to the directories containing the CSV files
dir_2_steps_rag = "results 2-steps RAG"
dir_common_rag = "results Common RAG"

# Function to read CSV files from a directory and add a 'rag_type' column
def read_and_label_csvs(directory, rag_type):
    dfs = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            try:
                df = pd.read_csv(filepath, encoding='utf-8')
                df['rag_type'] = rag_type
                columns_to_remove = ['score', 'prompt_tokens', 'augmented_prompt_tokens', 'results_tokens', 'prompt_cost', 'results_cost', 'total_cost']
                df = df.drop(columns=[col for col in columns_to_remove if col in df.columns])
                dfs.append(df)
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    return dfs

# Read and label CSV files from both directories
dfs_2_steps = read_and_label_csvs(dir_2_steps_rag, "2-steps RAG")
dfs_common = read_and_label_csvs(dir_common_rag, "Common RAG")

# Concatenate all DataFrames into a single DataFrame
all_dfs = dfs_2_steps + dfs_common

if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    
    # Reorder columns
    desired_order = ['model', 'attempt', 'temperature', 'top_p', 'rag_type', 'prompt', 'augmented_prompt', 'baseline', 'results']
    existing_columns = [col for col in desired_order if col in final_df.columns]
    final_df = final_df[existing_columns]

    print(final_df.head())
    print(final_df.rag_type.value_counts())
else:
    print("No CSV files found in the specified directories.")


                    model  attempt  temperature  top_p     rag_type  \
0  gpt-4o-mini-2024-07-18        1          0.1    0.1  2-steps RAG   
1  gpt-4o-mini-2024-07-18        1          0.1    0.5  2-steps RAG   
2  gpt-4o-mini-2024-07-18        1          0.1    0.9  2-steps RAG   
3  gpt-4o-mini-2024-07-18        1          1.0    0.1  2-steps RAG   
4  gpt-4o-mini-2024-07-18        1          1.0    0.5  2-steps RAG   

                                              prompt  \
0  Qual a descrição oficial do código NCM para a ...   
1  Qual a descrição oficial do código NCM para a ...   
2  Qual a descrição oficial do código NCM para a ...   
3  Qual a descrição oficial do código NCM para a ...   
4  Qual a descrição oficial do código NCM para a ...   

                                    augmented_prompt  \
0  \n    Você é um assistente especializado em re...   
1  \n    Você é um assistente especializado em re...   
2  \n    Você é um assistente especializado em re...   
3  \n    Voc

In [4]:
import os
from dotenv import load_dotenv
import openai
from tqdm import tqdm
import time

# Specify the path to your .env file
dotenv_path = "/mnt/4d4f90e5-f220-481e-8701-f0a546491c35/arquivos/projetos/.env"

# Load the .env file
load_dotenv(dotenv_path=dotenv_path)

# Access and store the environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")
model = 'gpt-4o-mini-2024-07-18'

In [ ]:
final_df['comparison_prompt'] # criar o prompt de comparação (Vinicius)

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import json

# Iterate over each row and make API call
output_filename = f"model scoring.csv"
final_df['evaluation'] = None

# Initialize progress bar
progress_bar = IntProgress(min=0, max=len(final_df), desc=f"Processing {model}")
display(progress_bar)

for index, row in final_df.iterrows():
    if index % 100 == 0 and index != 0:
        print("min. pause...")
        time.sleep(60)
    try:
        tmp_comparison_prompt = row['comparison_prompt']

        response = openai.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": tmp_comparison_prompt}],
            temperature=float(row['temperature']),
            top_p=float(row['top_p'])
        )

        # Extract and store the generated text
        metadata_str = response.choices[0].message.content.strip()
        if metadata_str.startswith("```json") and metadata_str.endswith("```"):
            metadata_str = metadata_str.strip("```").strip()
        if metadata_str.startswith("json"):
            metadata_str = metadata_str[4:].lstrip()
        metadata = json.loads(metadata_str)

        filters = {}
        if isinstance(metadata, dict):
            filters = {k: v for k, v in metadata.items() if v is not None}

        final_df.loc[index, 'evaluation'] = filters

    except openai.OpenAIError as e:
        print(f"Error processing row {index} for model {model}: {e}")
        final_df.loc[index, 'evaluation'] = f"Error: {e}"  # Store the error message
    except Exception as e:
        print(f"Unexpected error processing row {index} for model {model}: {e}")
        final_df.loc[index, 'evaluation'] = f"Error: {e}"

    # Update progress bar
    progress_bar.value += 1

# Save the updated DataFrame (optional)
final_df.to_csv(output_filename, index=False)